In [1]:
import pyspark.sql.functions as fn
from pyspark.sql import SparkSession
import pandas as pd

### First lets create some dataframes to work with
(We'll use pandas as an intermediary to avoid having to construct schema)

In [3]:
# Create a spark session
spark_session = SparkSession.builder.getOrCreate()

df_data = {'col_1': [1,2,3,4,5], 
           'col_2': [5,4,3,2,1],
           'col_3': ['a', 'b', 'c', 'd', 'e'],
           'col_4': ['one', 'two', 'one', 'two', 'one']
          }
df_pandas = pd.DataFrame.from_dict(df_data)
# create spark dataframe
df = spark_session.createDataFrame(df_pandas)
# let us also prepare the dataframe for use as an SQL table
df.createOrReplaceTempView("table")

left_data = {'left_1': [1,2,3,4,5], 
           'left_2': ['one', 'two', 'one', 'two', 'one']
          }
left_pandas = pd.DataFrame.from_dict(left_data)
# create spark dataframe
left_df = spark_session.createDataFrame(left_pandas)
# let us also prepare the dataframe for use as an SQL table
left_df.createOrReplaceTempView("left_table")

right_data = {'right_1': [5,4,3,2,1],
           'right_3': ['a', 'b', 'c', 'd', 'e']
          }
right_pandas = pd.DataFrame.from_dict(right_data)
# create spark dataframe
right_df = spark_session.createDataFrame(right_pandas)
# let us also prepare the dataframe for use as an SQL table
right_df.createOrReplaceTempView("right_table")


## Now we have our dataframes and sql tables let us use them to compare and contrast SQL and PySpark syntax

### Comparison of table description syntax

In [6]:
# pyspark
#df_pyspark = 
df.printSchema()
#df_pyspark.show()

# SQL
df_sql = spark.sql("DESCRIBE table")
df_sql.show()

root
-- col_1: long (nullable = true)
-- col_2: long (nullable = true)
-- col_3: string (nullable = true)
-- col_4: string (nullable = true)

+--------+---------+-------+
col_name|data_type|comment|
+--------+---------+-------+
 col_1| bigint| null|
 col_2| bigint| null|
 col_3| string| null|
 col_4| string| null|
+--------+---------+-------+

### Comparison of select syntax including aliasing

In [8]:
# pyspark
df_pyspark = df.select(fn.col("col_1").alias("f1"), fn.col("col_3").alias("f3"))
df_pyspark.show()

# SQL
df_sql = spark.sql("SELECT col_1 AS f1, col_3 AS f3 FROM table")
df_sql.show()

+---+---+
 f1| f3|
+---+---+
 1| a|
 2| b|
 3| c|
 4| d|
 5| e|
+---+---+

+---+---+
 f1| f3|
+---+---+
 1| a|
 2| b|
 3| c|
 4| d|
 5| e|
+---+---+

### Selection of distinct results

In [10]:
# pyspark
df_pyspark = df.select(fn.col("col_4")).distinct()
df_pyspark.show()

# SQL
df_sql = spark.sql("SELECT DISTINCT col_4 FROM table")
df_sql.show()

+-----+
col_4|
+-----+
 two|
 one|
+-----+

+-----+
col_4|
+-----+
 two|
 one|
+-----+

### Comparison of limiting the number of returns

In [12]:
# pyspark
df_pyspark = df.limit(2)
df_pyspark.show()

# SQL
df_sql = spark.sql("SELECT * FROM table LIMIT 2")
df_sql.show()

+-----+-----+-----+-----+
col_1|col_2|col_3|col_4|
+-----+-----+-----+-----+
 1| 5| a| one|
 2| 4| b| two|
+-----+-----+-----+-----+

+-----+-----+-----+-----+
col_1|col_2|col_3|col_4|
+-----+-----+-----+-----+
 1| 5| a| one|
 2| 4| b| two|
+-----+-----+-----+-----+

### Comparison of ordering syntax
Default ordering is ascending in each case

In [14]:
# pyspark
df_pyspark = df.orderBy("col_2")
df_pyspark.show()

# SQL
df_sql = spark.sql("SELECT * FROM table ORDER BY col_2")
df_sql.show()

+-----+-----+-----+-----+
col_1|col_2|col_3|col_4|
+-----+-----+-----+-----+
 5| 1| e| one|
 4| 2| d| two|
 3| 3| c| one|
 2| 4| b| two|
 1| 5| a| one|
+-----+-----+-----+-----+

+-----+-----+-----+-----+
col_1|col_2|col_3|col_4|
+-----+-----+-----+-----+
 5| 1| e| one|
 4| 2| d| two|
 3| 3| c| one|
 2| 4| b| two|
 1| 5| a| one|
+-----+-----+-----+-----+

In [15]:
# pyspark
df_pyspark = df.orderBy("col_1", ascending = False)
df_pyspark.show()

# SQL
df_sql = spark.sql("SELECT * FROM table ORDER BY col_1 DESC")
df_sql.show()

+-----+-----+-----+-----+
col_1|col_2|col_3|col_4|
+-----+-----+-----+-----+
 5| 1| e| one|
 4| 2| d| two|
 3| 3| c| one|
 2| 4| b| two|
 1| 5| a| one|
+-----+-----+-----+-----+

+-----+-----+-----+-----+
col_1|col_2|col_3|col_4|
+-----+-----+-----+-----+
 5| 1| e| one|
 4| 2| d| two|
 3| 3| c| one|
 2| 4| b| two|
 1| 5| a| one|
+-----+-----+-----+-----+

### Comparison of filter syntax

In [17]:
# PySpark
# note that all the following achieve the same result (pick the syntax you prefer)
df_pyspark = df.filter(df.col_1 > 3)
df_pyspark.show()
df_pyspark = df.filter(fn.col("col_1") > fn.lit(3))
df_pyspark.show()
df_pyspark = df.filter(df["col_1"] > fn.lit(3))
df_pyspark.show()
# we can even use operators
from operator import *
df_pyspark = df.filter(gt(df.col_1,3))
df_pyspark.show()

# SQL
df_sql = spark.sql("SELECT * FROM table WHERE col_1 > 3")
df_sql.show()

+-----+-----+-----+-----+
col_1|col_2|col_3|col_4|
+-----+-----+-----+-----+
 4| 2| d| two|
 5| 1| e| one|
+-----+-----+-----+-----+

+-----+-----+-----+-----+
col_1|col_2|col_3|col_4|
+-----+-----+-----+-----+
 4| 2| d| two|
 5| 1| e| one|
+-----+-----+-----+-----+

+-----+-----+-----+-----+
col_1|col_2|col_3|col_4|
+-----+-----+-----+-----+
 4| 2| d| two|
 5| 1| e| one|
+-----+-----+-----+-----+

+-----+-----+-----+-----+
col_1|col_2|col_3|col_4|
+-----+-----+-----+-----+
 4| 2| d| two|
 5| 1| e| one|
+-----+-----+-----+-----+

+-----+-----+-----+-----+
col_1|col_2|col_3|col_4|
+-----+-----+-----+-----+
 4| 2| d| two|
 5| 1| e| one|
+-----+-----+-----+-----+

### Comparison of group by syntax

In [19]:
# PySpark
# note that all the following achieve the same result (pick the syntax you prefer)
df_pyspark = df.groupBy("col_4").agg(fn.count("col_1"), fn.sum("col_2"))
df_pyspark.show()

# SQL
df_sql = spark.sql("SELECT col_4, COUNT(col_1), SUM (col_2) FROM table GROUP BY col_4")
df_sql.show()

+-----+------------+----------+
col_4|count(col_1)|sum(col_2)|
+-----+------------+----------+
 two| 2| 6|
 one| 3| 9|
+-----+------------+----------+

+-----+------------+----------+
col_4|count(col_1)|sum(col_2)|
+-----+------------+----------+
 two| 2| 6|
 one| 3| 9|
+-----+------------+----------+

### Comparison of table join syntax
Note that row order is not preserved in either case. Useful reference at http://www.learnbymarketing.com/1100/pyspark-joins-by-example/

### Inner join

In [22]:
# PySpark
# note that Pyspark  join defaults to inner join
df_pyspark = left_df.join(right_df, left_df.left_1 == right_df.right_1)
df_pyspark.show()

# SQL
df_sql = spark.sql("SELECT * FROM left_table INNER JOIN right_table ON left_table.left_1 = right_table.right_1")
df_sql.show()


+------+------+-------+-------+
left_1|left_2|right_1|right_3|
+------+------+-------+-------+
 5| one| 5| a|
 1| one| 1| e|
 3| one| 3| c|
 2| two| 2| d|
 4| two| 4| b|
+------+------+-------+-------+

+------+------+-------+-------+
left_1|left_2|right_1|right_3|
+------+------+-------+-------+
 5| one| 5| a|
 1| one| 1| e|
 3| one| 3| c|
 2| two| 2| d|
 4| two| 4| b|
+------+------+-------+-------+

### Outer join

In [24]:
# PySpark
# use the how keyword to specify join type
df_pyspark = left_df.join(right_df, left_df.left_1 == right_df.right_1, how = "outer")
df_pyspark.show()

# SQL - note the use of FULL
df_sql = spark.sql("SELECT * FROM left_table FULL OUTER JOIN right_table ON left_1 = right_1")
df_sql.show()


+------+------+-------+-------+
left_1|left_2|right_1|right_3|
+------+------+-------+-------+
 5| one| 5| a|
 1| one| 1| e|
 3| one| 3| c|
 2| two| 2| d|
 4| two| 4| b|
+------+------+-------+-------+

+------+------+-------+-------+
left_1|left_2|right_1|right_3|
+------+------+-------+-------+
 5| one| 5| a|
 1| one| 1| e|
 3| one| 3| c|
 2| two| 2| d|
 4| two| 4| b|
+------+------+-------+-------+

### Left join

In [26]:
# PySpark
# note that Pyspark  join defaults to inner join
# use & and | to represent and AND and OR conditions
df_pyspark = left_df.join(right_df, left_df.left_1 == right_df.right_1, how = "left")
df_pyspark.show()

# SQL
df_sql = spark.sql("SELECT * FROM left_table LEFT JOIN right_table ON left_1 = right_1")
df_sql.show()


+------+------+-------+-------+
left_1|left_2|right_1|right_3|
+------+------+-------+-------+
 5| one| 5| a|
 1| one| 1| e|
 3| one| 3| c|
 2| two| 2| d|
 4| two| 4| b|
+------+------+-------+-------+

+------+------+-------+-------+
left_1|left_2|right_1|right_3|
+------+------+-------+-------+
 5| one| 5| a|
 1| one| 1| e|
 3| one| 3| c|
 2| two| 2| d|
 4| two| 4| b|
+------+------+-------+-------+

### Cross join

In [28]:
# PySpark
# note that Pyspark  has a specific command for the cross join
df_pyspark = left_df.crossJoin(right_df)
df_pyspark.show()

# SQL
df_sql = spark.sql("SELECT * FROM left_table CROSS JOIN right_table")
df_sql.show()

+------+------+-------+-------+
left_1|left_2|right_1|right_3|
+------+------+-------+-------+
 1| one| 5| a|
 1| one| 4| b|
 1| one| 3| c|
 1| one| 2| d|
 1| one| 1| e|
 2| two| 5| a|
 2| two| 4| b|
 2| two| 3| c|
 2| two| 2| d|
 2| two| 1| e|
 3| one| 5| a|
 3| one| 4| b|
 3| one| 3| c|
 3| one| 2| d|
 3| one| 1| e|
 4| two| 5| a|
 4| two| 4| b|
 4| two| 3| c|
 4| two| 2| d|
 4| two| 1| e|
+------+------+-------+-------+
only showing top 20 rows

+------+------+-------+-------+
left_1|left_2|right_1|right_3|
+------+------+-------+-------+
 1| one| 5| a|
 1| one| 4| b|
 1| one| 3| c|
 1| one| 2| d|
 1| one| 1| e|
 2| two| 5| a|
 2| two| 4| b|
 2| two| 3| c|
 2| two| 2| d|
 2| two| 1| e|
 3| one| 5| a|
 3| one| 4| b|
 3| one| 3| c|
 3| one| 2| d|
 3| one| 1| e|
 4| two| 5| a|
 4| two| 4| b|
 4| two| 3| c|
 4| two| 2| d|
 4| two| 1| e|
+------+------+-------+-------+
only showing top 20 rows

### Concatenation of dataframes

In [30]:
# UNION

# PySpark
# note that Pyspark Union is a union all so use distinct to get union
df_pyspark = left_df.select("left_1").union(right_df.select("right_1")).distinct()
df_pyspark.show()

# SQL
df_sql = spark.sql("SELECT left_1 FROM left_table UNION SELECT Right_1 FROM right_table")
df_sql.show()

+------+
left_1|
+------+
 5|
 1|
 3|
 2|
 4|
+------+

+------+
left_1|
+------+
 5|
 1|
 3|
 2|
 4|
+------+

In [31]:
# UNION ALL

# PySpark
# note that PySpark union performs union all (PySpark unionAll is deprecated)
df_pyspark = left_df.select("left_1").union(right_df.select("right_1"))
df_pyspark.show()

# SQL
df_sql = spark.sql("SELECT left_1 FROM left_table UNION ALL SELECT Right_1 FROM right_table")
df_sql.show()

+------+
left_1|
+------+
 1|
 2|
 3|
 4|
 5|
 5|
 4|
 3|
 2|
 1|
+------+

+------+
left_1|
+------+
 1|
 2|
 3|
 4|
 5|
 5|
 4|
 3|
 2|
 1|
+------+

### Amending data in a table
Note that SQL ALTER and SET commands will not work on Pyspark dataframes because they are immutable, therefore you have to use PySpark's withColumn function and assign to a dataframe

In [33]:
# PySpark
df_pyspark = df.withColumn("col_4", fn.when(df.col_4 == 'two', 'changed').otherwise(df.col_4))
df_pyspark.show()

# SQL
# IMPORTANT - ALTER and SET do not work as dataframes are immutable - here is a fallback
df_sql = spark.sql("SELECT col_1, col_2, col_3, CASE WHEN col_4 = 'two' THEN 'changed' ELSE col_4 END AS col_4 FROM table")
df_sql.show()

# alternatively you can combine spark.sql and pyspark commands
df_sql = spark.sql("SELECT * FROM table").withColumn("col_4", fn.when(df.col_4 == 'two', 'changed').otherwise(df.col_4))
df_sql.show()

+-----+-----+-----+-------+
col_1|col_2|col_3| col_4|
+-----+-----+-----+-------+
 1| 5| a| one|
 2| 4| b|changed|
 3| 3| c| one|
 4| 2| d|changed|
 5| 1| e| one|
+-----+-----+-----+-------+

+-----+-----+-----+-------+
col_1|col_2|col_3| col_4|
+-----+-----+-----+-------+
 1| 5| a| one|
 2| 4| b|changed|
 3| 3| c| one|
 4| 2| d|changed|
 5| 1| e| one|
+-----+-----+-----+-------+

+-----+-----+-----+-------+
col_1|col_2|col_3| col_4|
+-----+-----+-----+-------+
 1| 5| a| one|
 2| 4| b|changed|
 3| 3| c| one|
 4| 2| d|changed|
 5| 1| e| one|
+-----+-----+-----+-------+

## Further reading
###https://docs.databricks.com/spark/latest/dataframes-datasets/introduction-to-dataframes-python.html
###https://spark.apache.org/docs/2.2.0/sql-programming-guide.html